In [3]:
from src.database.DataBase__singleton import MySQLConnector

db = MySQLConnector()
db.connect()

Database henryapp-base connection was made successfully


In [6]:
idxs_to_create = []

## Índice para mejorar joins entre sales y products
idx_prod_id = "CREATE INDEX idx_sales_product_id ON sales(product_id);"
idxs_to_create.append(idx_prod_id)

## Índice para mejorar joins entre sales y employees
idx_salesperson_id = "CREATE INDEX idx_sales_sales_person_id ON sales(sales_person_id);"
idxs_to_create.append(idx_salesperson_id)

## Índice para mejorar búsquedas por precio y nombre en products
idx_price_name ="CREATE INDEX idx_products_price_name ON products(price, name);"
idxs_to_create.append(idx_price_name)

## Índice para acelerar joins por employee_id
idx_employee_id = "CREATE INDEX idx_employees_id ON employees(employee_id);"
idxs_to_create.append(idx_employee_id)

for idx in idxs_to_create:
    db.query_insert_delete_update_triggers_idxs(typeof="Indice", query=idx)
    

Indice creado exitosamente
Indice creado exitosamente
Indice creado exitosamente
Indice creado exitosamente


In [16]:
# Top 3 productos más vendidos por cantidad total
sql_cte = '''
-- Busco los productos mas vendidos por cantidad total
WITH ProductSales AS (
    SELECT 
        p.id AS product_id,
        p.name AS product_name,
        SUM(s.quantity) AS total_quantity
    FROM sales s
    JOIN products p ON s.product_id = p.id
    GROUP BY p.id, p.name
),
-- Hago un ranking de esa CTE
RankedProducts AS (
    SELECT 
        product_id,
        product_name,
        total_quantity,
        RANK() OVER (ORDER BY total_quantity DESC) AS rank_position
    FROM ProductSales
)
-- Selecciono los 3 primeros del ranking
SELECT 
    product_id,
    product_name,
    total_quantity,
    rank_position
FROM RankedProducts
WHERE rank_position <= 3;
'''
db.query_alchemy(query=sql_cte)


,product_id,product_name,total_quantity,rank_position
0,156,Sprouts - Alfalfa,1907.0,1
1,177,Coconut - Shredded; Sweet,1896.0,2
2,174,Guinea Fowl,1875.0,3


In [17]:
## Los mejores 3 vendedores para el producto mas caro
sql_cte = '''
-- Obtengo el precio del producto mas caro
WITH MaxPrice AS (
    SELECT MAX(price) AS max_price FROM products
), 

-- Obtengo el producto, los datos del vendedor y la cantidad vendida para el producto mas caro
FilteredProducts AS (
    SELECT 
        p.name AS product_name,
        e.first_name AS employee_name, 
        e.last_name AS employee_lastname,
        COUNT(s.sales_id) AS quantity_sold
    FROM products p
    JOIN MaxPrice mp ON p.price = mp.max_price
    JOIN sales s ON s.product_id = p.id
    JOIN employees e ON e.employee_id = s.sales_person_id
    GROUP BY p.name, e.first_name, e.last_name
),

-- Hago un ranking de esa informacion
RankedSales AS (
    SELECT 
        product_name,
        employee_name,
        employee_lastname,
        quantity_sold,
        RANK() OVER (ORDER BY quantity_sold DESC) AS rank_position
    FROM FilteredProducts
)

-- Devuelvo los top tres de ese ranking
SELECT 
    product_name,
    employee_name,
    employee_lastname,
    quantity_sold,
    rank_position
FROM RankedSales
WHERE rank_position <= 3;
'''
db.query_alchemy(query=sql_cte)

,product_name,employee_name,employee_lastname,quantity_sold,rank_position
0,Bread - Calabrese Baguette,Desiree,Stuart,12,1
1,Bread - Calabrese Baguette,Christine,Palmer,9,2
2,Bread - Calabrese Baguette,Pablo,Cline,9,2
3,Bread - Calabrese Baguette,Janet,Flowers,9,2


In [18]:
## Última venta realizada por cada vendedor
sql_cte = ''' 
WITH SalesRanked AS (
    SELECT 
        s.sales_id,
        s.sales_person_id,
        s.product_id,
        s.total_price,
        s.sales_date,
        ROW_NUMBER() OVER (
            PARTITION BY s.sales_person_id 
            ORDER BY s.sales_date DESC
        ) AS rn
    FROM sales s
)
SELECT 
    sales_id,
    sales_person_id,
    product_id,
    total_price,
    sales_date
FROM SalesRanked
WHERE rn = 1;
'''
db.query_alchemy(query=sql_cte)


,sales_id,sales_person_id,product_id,total_price,sales_date
0,5356622,1,205,20.0,59:54.3
1,424688,2,77,36.0,59:57.2
2,4064414,3,208,6.0,59:56.8
3,6403777,4,124,60.0,59:57.4
4,2274474,5,353,80.0,59:59.3
5,1944938,6,15,144.0,59:59.2
6,734872,7,275,42.0,59:55.5
7,4101913,8,286,136.0,59:59.4
8,117578,9,393,45.0,59:59.0
9,5116075,10,390,50.0,59:58.0


In [ ]:
## Vista para obtener todos clientes con su país y ciudad
db.query_select(query="""
    CREATE VIEW view_customer_locations AS
    SELECT 
        c.customer_id,
        c.first_name,
        c.middle_initial,
        c.last_name,
        c.address,
        ci.city_name,
        co.country_name,
        co.country_code
    FROM customers c
    JOIN cities ci ON c.city_id = ci.city_id
    JOIN countries co ON ci.country_id = co.country_id;
""")

## Validamos que la vista se creo correctamente
db.query_select(query="SHOW FULL TABLES IN `henryapp-base` WHERE TABLE_TYPE = 'VIEW';")


[('view_customer_locations', 'VIEW')]

In [15]:
## Utilizamos la vista con alchemy
db.query_alchemy(query="SELECT * FROM view_customer_locations;")

,customer_id,first_name,middle_initial,last_name,address,city_name,country_name,country_code
0,1,Stefanie,Y,Frye,97 Oak Avenue\r,oklahoma,united states,AR\r
1,2,Sandy,T,Kirby,52 White First Freeway\r,pittsburgh,united states,AR\r
2,3,Lee,T,Zhang,921 White Fabien Avenue\r,houston,united states,AR\r
3,4,Regina,S,Avery,75 Old Avenue\r,cleveland,united states,AR\r
4,5,Daniel,S,Mccann,283 South Green Hague Avenue\r,buffalo,united states,AR\r
...,...,...,...,...,...,...,...,...
98754,98755,Yvette,C,Campos,945 Oak Parkway\r,mesa,united states,AR\r
98755,98756,Angelo,X,Mc Millan,99 Fabien Street\r,charlotte,united states,AR\r
98756,98757,Shari,J,Prince,791 Milton Drive\r,sacramento,united states,AR\r
98757,98758,Stuart,K,Cameron,149 Clarendon Road\r,nashville,united states,AR\r
